In [ ]:
# 기사 목록

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")

# Keywords to search
keywords = [
    "검색어",
]

# 검색 기간 설정
start_date = "2024.06.01"
end_date = "2024.08.31"

collected_urls = set()

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

def fetch_search_results(keyword, start_date, end_date):
    """지정된 검색어와 날짜 범위로 한겨레에서 기사 스크래핑."""
    base_url = "https://search.hani.co.kr/search/newslist?searchword="
    results = []
    page = 1  # 페이지 시작 번호 설정

    while True:
        print(f"{keyword} {page} 페이지 수집 중...")
        search_url = (
            f"{base_url}{urllib.parse.quote(keyword)}"
            f"&startdate={start_date}&enddate={end_date}&page={page}&sort=desc"
        )
        driver.get(search_url)

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul.article-list li"))
            )
        except Exception as e:
            print(f"페이지 로딩 중 오류 발생: {e}")
            break

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기사 목록 가져오기
        article_list = soup.find_all("ul", class_="article-list")
        if not article_list:  # 더 이상 기사가 없으면 종료
            break

        # 기사 정보 추출
        for ul in article_list:
            articles = ul.find_all("article")
            for item in articles:
                try:
                    # 'a' 태그를 찾아 제목과 URL을 가져옴
                    title_tag = item.find("a", class_="flex-inner")
                    title = title_tag.find("strong").text.strip() if title_tag else ""
                    url = title_tag["href"] if title_tag else ""

                    # 중복 URL 체크
                    if url in collected_urls:
                        continue  # 중복된 URL은 무시
                    collected_urls.add(url)  # 새로운 URL 저장

                    # 날짜 추출
                    date_element = item.find("span", class_="article-date")
                    date = date_element.text.strip().split(" ")[0] if date_element else ""

                    # 결과 리스트에 추가
                    results.append({
                        'media': '한겨레',
                        'title': title,
                        'date': date,
                        'section': '',
                        'article_type': '',
                        'writer': '',
                        'page': '',
                        'url': url,
                        'content': ''  # 컨텐츠 정보 없음
                    })

                except Exception as e:
                    print(f"기사 처리 중 오류 발생: {e}")

        page += 1  # 다음 페이지로 이동

    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df = df[~df['url'].str.contains('h21', case=False, na=False)] #주간지 제외
    df.to_excel(filename, index=False)

all_results = []

for keyword in keywords:
    results = fetch_search_results(keyword, start_date, end_date)
    all_results.extend(results)

# 결과를 엑셀 파일로 저장
save_to_excel(all_results, f'1한겨레 {start_date}~{end_date}.xlsx')

# 드라이버 종료
driver.quit()


In [ ]:
# 본문 추출
# 서브타이틀 x

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문을 스크랩하는 함수 정의
def get_content_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        article_body = soup.find('div', class_="arti-txt")
        if not article_body:
            article_body = soup.find('div', class_='article-text')

        if article_body:
            # 제거할 클래스 목록
            classes_to_remove = [
                'photo-box', 'ad-pc', 'ad-box', 'bn-promotion-support',
                'ArticleDetailContent_imageContainer___o_gm',
                'ArticleDetailAudioPlayer_wrap__vbjJ_',
                'ArticleDetailContent_adWrap__xYVGB', 'screen-out'
            ]

            # 각 클래스에 해당하는 태그를 찾아 제거
            for class_name in classes_to_remove:
                for tag in article_body.find_all('div', class_=class_name):
                    tag.decompose()
            
            content = article_body.get_text(separator='\n', strip=True)
          
        else:
            content = ""
        return content
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문 내용을 스크랩하여 content 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content = get_content_from_url(row['url'])
    if content.startswith('='):
        content = "'" + content      
    df.at[index, 'content'] = content

        # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['url'].str.contains('h21', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")
      
         